# Pick and Place

In this notebook, we will show you how to realize a simple pick and place example using Reachy and a Brunel Hand made by Open Bionics.

To realize this application, several steps must be followed:

* First, we put the robot in a **rest position**.
* Then, we **record by demonstration** the trajectory from this position to the **pick position**.
* We **close the hand** on the object.
* The record by demonstration the trajectory to the **place position**.
* We **open the hand** to release the object.
* Finally, we go back to our rest position.

We will describe each of them below.

### Prepare the setup

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import time

First, we'll connect to our robot and specify the port for the hand we are using.

In [ ]:
from reachy import Leachy

leachy = Leachy(brunel_hand='/dev/tty.usbmodem141411')

Define your rest position.

In [ ]:
rest_position = {
    'l_shoulder_pitch': -7,
    'l_shoulder_roll': 5,
    'l_arm_yaw': 0,
    'l_elbow_pitch': -35,
    'l_forearm_yaw': 20,
    'l_wrist_pitch': -30,
}

In [ ]:
def goto_rest(dur=2.0, wait=True):
    for m in leachy.motors:
        m.compliant = False

    leachy.goto_position(rest_position, duration=dur, wait=wait)

Check that everything is ok.

In [ ]:
goto_rest()

## Pick the object

We will record the trajectory to the pick position by demonstration, meaning a human user will directly move the arm of the robot and we will record the present position of its motor.

First, let's put reachy in compliant mode, where it can be freely moved.

In [ ]:
leachy.compliant = True

Now we will define a record function that lets you move the robot and record its present position for a predetermined duration (expressed in sec).

In [ ]:
def record_trajectory(duration, record_freq=50.0):
    t0 = time.time()
    
    traj = []
    while time.time() - t0 < duration:
        traj.append([m.present_position for m in leachy.motors])
        time.sleep(1.0 / record_freq)
    
    return np.array(traj)

Now, move the robot to the pick location. You have 5 seconds!

In [ ]:
pick_traj = record_trajectory(5.0)
print(len(pick_traj), 'position recorded')

You can plot the trajectory to check that it went well.

In [ ]:
plt.plot(pick_traj)
plt.legend([m.name for m in leachy.motors], loc='upper right')

We now need to define a function that will be used to replay a trajectory.

In [ ]:
def replay_trajectory(traj, play_freq=50.0):
    for m in leachy.motors:
        m.compliant = False
    
    for pos in traj:
        goal = {
            m.name: p
            for m, p in zip(leachy.motors, pos)
        }
        leachy.goto_position(goal, 1.0 / play_freq)
        time.sleep(1.0 / play_freq)

Let's try our new trajectory by first going to the rest position, then to the pick position.

In [ ]:
goto_rest()
replay_trajectory(pick_traj)

You can re-record your pick trajectory until you are satisfied with the result.

And finally, to end the pick part of our application we need to close the hand to grasp the object.

In [ ]:
leachy.hand.close()

## Place the object

Now that we have picked the object, we want to place it in the place location. We will proceed as before. First, we record the place trajectory by demonstration, then check if the robot replays it correctly and finally open the hand to release the object.

First, we record the place trajectory. As before, you have 5 seconds.

In [ ]:
leachy.compliant = True

place_traj = record_trajectory(5.0)
print(len(place_traj), 'position recorded')

Then, we replay our trajectory. To insure that we don't do unexpected movement, we will make sure to start from the first position of our trajectory.

In [ ]:
leachy.compliant = False

start_of_trajectory = {
    m.name: p
    for m, p in zip(leachy.motors, place_traj[0])
}

leachy.goto_position(start_of_trajectory, 2.0, wait=True)

And now we play our trajectory.

In [ ]:
replay_trajectory(place_traj)

If you are satisfied with the behavior and the end position of the trajectory, you can release the object via:

In [ ]:
leachy.hand.open()

That's it for the place part!

## Putting everything together

In [ ]:
def pick_and_place():
    goto_rest()
    
    # Pick
    replay_trajectory(pick_traj)
    time.sleep(1.0) # Make sure we are stabilized
    leachy.hand.close()
    
    # Place
    replay_trajectory(place_traj)
    time.sleep(1.0)
    leachy.hand.open()

You can now try your new application.

In [ ]:
pick_and_place()

If everything looks ok, you can run it in a loop.

In [ ]:
for _ in range(10):
    pick_and_place()

## To go further

Even if it's working, this behavior could still be improved. For instance, you could:

* edit the trajectory to smooth the transition
* use other control method to replay the trajectory to smooth acceleration at the beginning and end of the trajectory
* check the motor temperature to avoid overheating